Creating the dictionary from the train data


In [2]:
from sklearn.feature_extraction.text import CountVectorizer

class CreateDictionary():
    def __init__(self,max_features=None,max_df=1.0,min_df=1):
        self.vectorizer=CountVectorizer(max_features=max_features,max_df=max_df,min_df=min_df)
        self.vocab=None
    
    def fit(self,text):
        self.vectorizer.fit(text)
        self.vocab=self.vectorizer.vocabulary_

def text_to_vector(text,vocabulary):
    vector=[0]*len(vocabulary)
    for word in text.split():
        if word in vocabulary:
            vector[list(vocabulary.keys()).index(word)]=1
    return vector
  

Reading the data , and creating the vectors


In [39]:
import os

class DataManipulation():
    def __init__(self,positive_reviews_dir,negative_reviews_dir,dictionary):
        self.negative_reviews_dir=negative_reviews_dir
        self.positive_reviews_dir=positive_reviews_dir
        self.positive_reviews_files = os.listdir(positive_reviews_dir)
        self.negative_reviews_files= os.listdir(negative_reviews_dir)
        self.dictionary=dictionary
    
    def cdictionary(self):
        text = []
        dictionary=CreateDictionary(1000,0.8,4)
        for file in self.positive_reviews_files:
            with open(os.path.join(self.positive_reviews_dir, file), "r", encoding='utf-8-sig') as f:
                text.append(f.read())
        for file in self.negative_reviews_files:
            with open(os.path.join(self.negative_reviews_dir, file), "r", encoding='utf-8-sig') as f:
                text.append(f.read())
        dictionary.fit(text)
        self.dictionary=dictionary
        return dictionary

    def vector(self):
        positive=[]
        for filename in self.positive_reviews_files:
            if filename.endswith(".txt"):
                file_path = os.path.join(self.positive_reviews_dir, filename)
                with open(file_path, 'r',encoding='utf-8') as file:
                    text = file.read()
                    positive.append(text_to_vector(text,self.dictionary.vocab))
        negative=[]
        for filename in self.negative_reviews_files:
            if filename.endswith(".txt"):
                file_path = os.path.join(self.negative_reviews_dir, filename)
                with open(file_path, 'r',encoding='utf-8') as file:
                    text = file.read()
                    negative.append(text_to_vector(text,self.dictionary.vocab)) 
        return negative,positive








Multinomial Naive Bayes

In [1]:



class MultinomialNaiveBayes:
    def __init__(self):
        self.class_word_freq = {}
        self.class_priors = {}
        
    def fit(self, X, y):
        for xi, label in zip(X, y):
            if label not in self.class_word_freq:
                self.class_word_freq[label] = [0] * len(xi)
            for i, word in enumerate(xi):
                if word == 1:
                    self.class_word_freq[label][i] += 1
        
       
        count=0
        for (yi) in y:
            if (yi==1):
                count=count+1
        self.class_priors = {0:(len(y)-count)/(len(y)),1:count/len(y)}
        
    def predict(self, X):
        predictions = []
        for xi in X:
            posteriors = []
            for label, word_freq in self.class_word_freq.items():
                likelihood = 1
                for i, word in enumerate(xi):
                    if word == 1:
                        likelihood *= word_freq[i] / sum(word_freq)
                prior = self.class_priors[label]
                posterior = likelihood * prior
                posteriors.append((posterior, label))
            predictions.append(sorted(posteriors, reverse=True)[0][1])
        return predictions










Tο directory και η δημιουργία του λεξικού 

In [29]:

positive_reviews_dir_train = '../train/pos'
negative_reviews_dir_train='../train/neg'
Xtr=DataManipulation(positive_reviews_dir_train,negative_reviews_dir_train,None)
positive,negative=Xtr.vector()
X_train=negative+positive
Y_train=[1]*len(negative)+[0]*len(positive)





In [ ]:
X_train=negative+positive
Y_train=[1]*len(negative)+[0]*len(positive)
positive_reviews_dir_test='../test/pos'
negative_reviews_dir_test='../test/neg'
Xts=DataManipulation(positive_reviews_dir_test,negative_reviews_dir_test,Xtr.dictionary)
positivet,negativet=Xts.vector()
X_test=negativet+positivet
Y_test=[1]*len(negativet)+[0]*len(positivet)
   

In [67]:
Y_test=[1]*len(negativet)+[0]*len(positivet)
mnb=MultinomialNaiveBayes()
mnb.fit(X_train,Y_train)
predictions=mnb.predict(X_test)



Το report 

In [111]:
from sklearn.metrics import classification_report

print(classification_report(Y_test, mnb.predict(X_test)))
print(classification_report(Y_train,mnb.predict(X_train)))

              precision    recall  f1-score   support

           0       0.81      0.75      0.78     12500
           1       0.77      0.82      0.79     12500

    accuracy                           0.78     25000
   macro avg       0.79      0.78      0.78     25000
weighted avg       0.79      0.78      0.78     25000

              precision    recall  f1-score   support

           0       0.82      0.74      0.78     12500
           1       0.76      0.84      0.80     12500

    accuracy                           0.79     25000
   macro avg       0.79      0.79      0.79     25000
weighted avg       0.79      0.79      0.79     25000

